

# Propaganda, start the `spark` session

> For SQL users, Spark SQL provides state-of-the-art SQL performance and maintains compatibility with Shark/Hive. In particular, like Shark, Spark SQL supports all existing Hive data formats, user-defined functions (UDF), and the Hive metastore.

> For Spark users, Spark SQL becomes the narrow-waist for manipulating (semi-) structured data as well as ingesting data from sources that provide schema, such as JSON, Parquet, Hive, or EDWs. It truly unifies SQL and sophisticated analysis, allowing users to mix and match SQL and more imperative programming APIs for advanced analytics.

> For open source hackers, Spark SQL proposes a novel, elegant way of building query planners. It is incredibly easy to add new optimizations under this framework.

> Internally, a structured query is a Catalyst tree of (logical and physical) relational operators and expressions.




In [ ]:
# import the usual suspects
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import sys
import timeit

%matplotlib inline
import seaborn as sns

sns.set_context("notebook", font_scale=1.2)

During the session, we will use classes and functions exported by `pyspark`


In [ ]:
# spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.catalog import Catalog
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType

Start the `SparkSession`

In [ ]:
conf = SparkConf().setAppName("Spark SQL Illustrations")
sc = SparkContext(conf=conf)

spark = (SparkSession
    .builder
    .appName("Spark SQL")
    .getOrCreate()
)

US Baby Names 1880-2017
=======================


Description
: US baby names provided by the SSA. 

This dataset contains all names used
for at least 5 children of either sex during a year. 


The file is made of `1924665` lines and  4 columns.

```
|-- name: string (nullable = true)
    |-- n: integer (nullable = true)
    |-- sex: string (nullable = true)
    |-- year: integer (nullable = true)
```

Each row indicates for a given name, sex, and year the number of babies 
of the given sex who were given that name during the given year. Names 
with less than 5 occurrences during the year were note recorded. 

|    name|  n|sex|year|
|:--------|:---:|:---:|:----:|
|  Emilia|112|  F|1985|
|   Kelsi|112|  F|1985|
|  Margot|112|  F|1985|
|  Mariam|112|  F|1985|
|Scarlett|112|  F|1985|

First, we download the data if it's not there yet

In [ ]:
import requests, zipfile, io
from pathlib import Path

path = Path('babynames_short.csv')
if not path.exists():
    url = "https://stephanegaiffas.github.io/big_data_course/data/babynames_short.csv.zip"
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path='./')

Load `babynames` from a `csv` file

In [ ]:
df_sp = spark.read\
             .format('csv')\
             .option("header", "true")\
             .option("mode", "FAILFAST")\
             .option("inferSchema", "true")\
             .option("sep", ",")\
             .load("babynames_short.csv")

df_sp.printSchema()


Ensure that the dataframe has the following schema:

    root
        |-- name: string (nullable = true)
        |-- n: integer (nullable = true)
        |-- sex: string (nullable = true)
        |-- year: integer (nullable = true)




In [ ]:
df_sp=df_sp.withColumn('year',df_sp.year.cast('int'))

df_sp.printSchema()

SQL versus spark-Dataframe API
=================================

>  Dataset API vs SQL

> Spark SQL supports two "modes" to write structured queries: Dataset API and SQL. SQL Mode is used to express structured queries using SQL statements using SparkSession.sql operator, expr standard function and spark-sql command-line tool.

> Some structured queries can be expressed much easier using Dataset API, but there are some that are only possible in SQL. In other words, you may find mixing Dataset API and SQL modes challenging yet rewarding.

> What is important, and one of the reasons why Spark SQL has been so successful, is that there is no performance difference between the modes. Whatever mode you use to write your structured queries, they all end up as a tree of Catalyst relational data structures. And, yes, you could consider writing structured queries using Catalyst directly, but that could quickly become unwieldy for maintenance (i.e. finding Spark SQL developers who could be comfortable with it as well as being fairly low-level and therefore possibly too dependent on a specific Spark SQL version).

Warmup:  compute the 10 most popular names given to babies in year 2000.
======================================================================

## Using `spark.sql()`

In order to use mode `sql`, create a temporary view from the `DataFrame`.

1. What are temporary views made of?
1. Are there other kind of views in spark's world?

In [ ]:
Catalog(spark).listTables()

In [ ]:
# TODO: 
df_sp.createOrReplaceTempView("babynames")
Catalog(spark).listTables()


## A query is a plain SQL query embodied in a string.



In [ ]:
query = """SELECT name FROM babynames ORDER BY n DESC LIMIT 10"""
spark.sql(query).show()


> This phrasing is not consistent with the DRY principle. Fix this using formatted strings.

## Using the dataframe/dataset API

This can also be done using Spark SQL API.

### Pedestrian approach

1. First select `10` most popular names for girls in year `2000`, define `spark` dataframe
`top10_2000_f`.
1. Does the definition of `top10_2000_f` involve _transformations_, _actions_ or both?
1. What is the type of the result returned by `top10_2000_f.take(2)`? the type of elements of the result?



In [ ]:
# top10_2000_f = TODO:
top10_2000_f = df_sp.where('sex == "F"').where('year == 2000').orderBy(df_sp.n.desc()).limit(10)

In [ ]:
top10_2000_f.show()


1. Do the same thing for boys.



In [ ]:
# top10_2000_m = TODO:
top10_2000_m = df_sp.where('sex == "M"').where('year == 2000').orderBy(df_sp.n.desc()).limit(10)

In [ ]:
top10_2000_m.show()


1. Compute the _union_ of the two spark dataframes. Store the result in
dataframe `top10_2000`



In [ ]:
# top10_2000 = TODO:
top10_2000=top10_2000_f.union(top10_2000_m)
top10_2000.show()


### Do it again, complying  with DRY principle



In [ ]:
# TODO:
def dry_top10_2000(gender):
    return df_sp.where('sex == "'+gender+'"').where('year == 2000').orderBy(df_sp.n.desc()).limit(10)

top10_2000=dry_top10_2000('F').union(dry_top10_2000('M'))
top10_2000.show()




Name portfolio through ages
===========================

1. Compute for each year and sex the number of distinct names given that year.



In [ ]:
# nb_names_year_sex = TODO: 
query="""SELECT year, sex, count(name) as n_distinct 
FROM babynames 
GROUP BY year, sex 
ORDER BY year,sex """
spark.sql(query).show(5)

In [ ]:
nb_names_year_sex = df_sp.groupBy(col('year'),col('sex'))\
    .agg(fn.count(col("name"))\
    .alias('nb_distinct'))\
    .orderBy(col('year'),col('sex'))
#or
nb_names_year_sex = df_sp.groupBy([df_sp.year,df_sp.sex]).count()
nb_names_year_sex.show(5)


1. Plot the evolution of the number of distinct names as a function of `year`.
Use some aesthetics to distinguish sexes.




In [ ]:
# TODO:
nb_names_year_sex_pd=nb_names_year_sex.toPandas()
sns.set()
g=sns.lineplot(x="year",y="count",hue="sex",style="sex",data=nb_names_year_sex_pd)
g.set_title("number of distinct names with more then 5 occurences")
plt.ylabel("Number of distinct given names")
plt.show(g)



Assessing popularity through time
=================================

1. For each year and sex, compute the total number of births
1. Plot the evolution of the sex ratio over time
1. For each year, sex, and name compute the percentage of newborns
given that name for that given year.


> Use `Window` functions.



In [ ]:
# TODO:
df_sex_ratio=df_sp.groupBy([df_sp.year,df_sp.sex]).agg(fn.sum(col("n")).alias("total")).orderBy(df_sp.year)
df_sex_ratio.show(5)


In [ ]:
df_sex_ratio=df_sex_ratio.groupBy("year")\
    .pivot("sex")\
    .sum('total')\
    .withColumn("sex_ratio",fn.round(col("M")/col("F"),2))
df_sex_ratio.show()

In [ ]:
# %%
# TODO: plot sex ratio
# %%
df_sex_ratio_pd=df_sex_ratio.toPandas()
sns.set()
g=sns.lineplot(x="year",y="sex_ratio",data=df_sex_ratio_pd)
g.set_title("number of distinct names with more then 5 occurences")
plt.plot(np.linspace(1880,2020,141),np.array([.52/.48]*141),'r-')
plt.ylabel("Sex ratio")
plt.show(g)

In [ ]:

window_23= Window.partitionBy('year')

total_by_year=fn.sum('n').over(window_23)

df_pourcentage = df_sp.withColumn('total',total_by_year)\
    .withColumn('pourcentage',fn.round(col("n")/col("total"),8))\
    .orderBy('pourcentage')

df_pourcentage.show()


1. Compute for each year, sex and name  the `row_number`, `rank`, and `dense_rank`
of the name within that year and sex category, when names are sorted by increasing popularity.



In [ ]:
# TODO:
window_24 = Window.partitionBy('year','sex')\
    .orderBy(df_sp.n)

row_number = fn.row_number().over(window_24)
rank = fn.rank().over(window_24)
dense_rank = fn.dense_rank().over(window_24)

df_sp.withColumn('row number',row_number).withColumn('rank',rank).withColumn('dense_rank',dense_rank).show()





Evolution of top popular names through the century
==================================================


1. For each sex, select the ten most popular names in year 2000, and plot the proportion
of newborns given that name over time. Take into account that some names might have
zero occurrence during certain years.



In [ ]:
top10_2000_f_names=top10_2000_f.select('name')
top10_2000_m_names=top10_2000_m.select('name')

over_time_top10_2000_f_names = top10_2000_f_names.join(df_sp.where('sex = "F"'),'name','left')
over_time_top10_2000_m_names = top10_2000_m_names.join(df_sp.where('sex = "M"'),'name','left')

over_time_top10_2000_names = over_time_top10_2000_f_names.union(over_time_top10_2000_m_names)

df_proportion_top10 = over_time_top10_2000_names.withColumn('total',total_by_year)\
    .withColumn('proportion',fn.round(col("n")*100/col("total"),4))\
    .orderBy('year')

df_proportion_top10_pd=df_proportion_top10.toPandas()
sns.set()
g=sns.lineplot(x="year",y="proportion",hue="name",data=df_proportion_top10_pd)
g.set_title("evolution of top 10 names in year 2000 over time")
plt.ylabel("proportion")
plt.show(g)



1. Use `explain()` to determine the joining strategy used by spark.


Plot  the popularity of each of the top ten achievers from year 2000 with respect to time
==================================================================================




In [ ]:
# %%
# TODO:

top10_2000_names_count=df_sp.groupBy('name','year')\
    .agg(fn.sum('n').alias('n_total'))\
    .where('year = 2000')\
    .orderBy(col("n_total").desc()).limit(10)

top10_2000_names = top10_2000_names_count.select('name')

over_time_top10_2000_names = top10_2000_names.join(df_sp.groupBy('name','year')\
                                                .agg(fn.sum('n').alias('n_total')),'name','left')

over_time_top10_2000_names_pd =over_time_top10_2000_names.toPandas()
sns.set()
g=sns.lineplot(x="year",y="n_total",hue="name",data=over_time_top10_2000_names_pd)
g.set_title("the popularity of each of the top ten achievers from year 2000 with respect to time")
plt.ylabel("popularity")
plt.show(g)
# %%



Plot the total popularity of the top ten achievers from year 2000 with respect to time
==================================================================================




In [ ]:
# %%
# TODO:
total_popularity=over_time_top10_2000_names.groupBy('year').agg(fn.sum('n_total').alias('total')).orderBy('year')

total_popularity_pd =total_popularity.toPandas()
sns.set()
g=sns.lineplot(x="year",y="total",data=total_popularity_pd)
g.set_title("the total popularity of the top ten achievers from year 2000 with respect to time")
plt.ylabel("popularity")
plt.show(g)
# %%



Plot lorenz curves
=====================

Every year, the name counts define a discrete probability distribution.
This distribution, just as income or wealth distribution,
is (usually) far from being uniform. We want to assess how uneven it is.
We use the tools developed in econometrics.

Without loss of generality, that we handle a distribution over $1, \ldots, n$
where $n$ is the number of distinct names given during a year.
We assume that frequencies $p_1, p_2, \ldots, p_n$ are given in ascending order.

The Lorenz function maps $[0, 1] \to [0, 1]$.
$$L(x) = \sum_{i=1}^{\lfloor nx \rfloor} p_i$$.

1. Design a query that adds a column "lorenz" to the dataframe , and for each
row computes the value of the Lorenz function defined by `year`  and `sex`.




In [ ]:
# TODO:

window_10 = Window.partitionBy('year','sex').orderBy('n').rangeBetween(Window.unboundedPreceding, 0)


df_lorenz=df_pourcentage.withColumn('lorenz',fn.sum('pourcentage').over(window_10))

df_lorenz.show()



1. Design a function that takes as input a `year` and plots the Lorenz curve
for that year for both sexes.



In [114]:
# %%
# TODO:
def lorenz_curve(year):
    df_lorenz_year=df_lorenz.where('year='+year)
    df_lorenz_year_pd =df_lorenz_year.toPandas()
    sns.set()
    g=sns.lineplot(x="lorenz",y="n",data=df_lorenz_year_pd)
    g.set_title("the total popularity of the top ten achievers from year 2000 with respect to time")
    plt.xlabel("popularity")
    plt.show(g)
lorenz_curve('2000')
# %%

KeyboardInterrupt: 


Gini index
==========

The [Gini index](https://en.wikipedia.org/wiki/Gini_coefficient) is twice the surface of the area comprised between curves $y=x$
and $y=L(x)$.

Choose a formula that allows you to compute it efficiently.

$$G={\frac {2\sum _{i=1}^{n}iy_{i}}{n\sum _{i=1}^{n}y_{i}}}-{\frac {n+1}{n}}.$$

1. Design a query that computes the Gini index of the `babynames` distribution
for every `year` and `sex`.

1. Plot Gini index over time




In [ ]:
# TODO:
df_gini_10 = df_pourcentage.withColumn('preGini',col("n")* 2 /(col("n")*col("total")))

df_gini = df_gini_10.groupBy([df_gini_10.year,df_gini_10.sex])\
.agg(fn.sum(col("preGini")).alias("gini")).orderBy(df_gini_10.year)

df_gini.show()

In [ ]:
# %%
df_gini_pd=df_gini.toPandas()
sns.set()
g=sns.lineplot(x="year",y="gini",hue="sex",style="sex",data=df_gini_pd)
g.set_title("indice gini des nom de nouveau nee 1880-2020")
plt.ylabel("indice gini")
plt.show(g)
# %%




Close the door, leave work area clean
=====================================


In [ ]:
spark.stop()